In [2]:
import torch
import pandas as pd
import numpy as np
import pytorch_transformers

folder = 'd:\git\project1\data'

In [3]:
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

model = model_class.from_pretrained(pretrained_weights)

In [4]:
%%time
tokens_padded = pd.read_csv(f'{folder}/tweets_change_padded.csv')
tokens_padded

Wall time: 13.5 s


,retweets,date,time,change_1_direct,change_3_direct,change_7_direct,change_30_direct,change_90_direct,0,1,...,90,91,92,93,94,95,96,97,98,99
0,7.0,1558465200,45736,1.0,1.0,1.0,1.0,1.0,101,2190,...,0,0,0,0,0,0,0,0,0,0
1,68.0,1558897200,29586,1.0,1.0,1.0,1.0,1.0,101,2978,...,0,0,0,0,0,0,0,0,0,0
2,4.0,1558810800,75329,1.0,1.0,1.0,1.0,1.0,101,1002,...,0,0,0,0,0,0,0,0,0,0
3,85.0,1556737200,63389,1.0,1.0,0.0,1.0,1.0,101,16770,...,0,0,0,0,0,0,0,0,0,0
4,8.0,1558897200,5857,1.0,1.0,1.0,1.0,1.0,101,2978,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807251,15.0,1574449200,23952,0.0,0.0,0.0,0.0,0.0,101,100,...,0,0,0,0,0,0,0,0,0,0
807252,50.0,1574449200,52538,0.0,0.0,0.0,0.0,0.0,101,2057,...,0,0,0,0,0,0,0,0,0,0
807253,940.0,1574017200,36668,1.0,0.0,0.0,1.0,0.0,101,8819,...,2050,2620,102,0,0,0,0,0,0,0
807254,24.0,1574449200,55448,0.0,0.0,0.0,0.0,0.0,101,1002,...,0,0,0,0,0,0,0,0,0,0


In [5]:
tokens_padded.iloc[:,8:]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,101,2190,4328,2595,2121,2038,2042,8243,2011,1996,...,0,0,0,0,0,0,0,0,0,0
1,101,2978,3597,2378,3976,4978,1002,1022,1010,6109,...,0,0,0,0,0,0,0,0,0,0
2,101,1002,2980,1002,2980,1002,2980,100,100,100,...,0,0,0,0,0,0,0,0,0,0
3,101,16770,1024,1013,1013,1056,1012,2522,1013,12997,...,0,0,0,0,0,0,0,0,0,0
4,101,2978,3597,2378,2089,10216,2000,2531,2243,2021,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807251,101,100,12919,2015,7816,2000,21573,2978,3597,2378,...,0,0,0,0,0,0,0,0,0,0
807252,101,2057,2024,3565,3407,2000,14970,2008,1056,2480,...,0,0,0,0,0,0,0,0,0,0
807253,101,8819,2003,2085,2330,2005,1996,5221,3795,18411,...,2050,2620,102,0,0,0,0,0,0,0
807254,101,1002,3263,1001,3477,12952,2030,1001,2978,3597,...,0,0,0,0,0,0,0,0,0,0


In [6]:
%%time
padded = np.array(tokens_padded.iloc[:,8:])
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

input_ids

Wall time: 2.56 s


tensor([[ 101, 2190, 4328,  ...,    0,    0,    0],
        [ 101, 2978, 3597,  ...,    0,    0,    0],
        [ 101, 1002, 2980,  ...,    0,    0,    0],
        ...,
        [ 101, 8819, 2003,  ...,    0,    0,    0],
        [ 101, 1002, 3263,  ...,    0,    0,    0],
        [ 101, 3407, 1001,  ...,    0,    0,    0]])

In [7]:
def save_last_hidden(start, count):
    start_index = start*count
    end_index = (start+1)*count
    last_hidden_states = model(input_ids[start_index:end_index, :], attention_mask=attention_mask[start_index:end_index, :])
    last_hidden_states_df = pd.DataFrame(last_hidden_states[0][:,0,:].numpy())
    #print(np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']]))
    last_hidden_states_df['retweet'] = np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']])
    last_hidden_states_df['time'] = np.array(tokens_padded.loc[start_index:end_index-1, ['time']])
    last_hidden_states_df['date'] = np.array(tokens_padded.loc[start_index:end_index-1, ['date']])
    last_hidden_states_df['change_1_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_1_direct']])
    last_hidden_states_df['change_3_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_3_direct']])
    last_hidden_states_df['change_7_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_7_direct']])
    last_hidden_states_df['change_30_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_30_direct']])
    last_hidden_states_df['change_90_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_90_direct']])
    #print(last_hidden_states_df)
    last_hidden_states_df.to_csv(f'{folder}/last_hs_tweets_ch_{start_index}.csv', index=False)

In [11]:
%%time
rows_count = input_ids.shape[0]
chank_size = 1000
start = 251
for chank_num in range (start, (int)(rows_count/chank_size)+1):
    print(chank_num)
    with torch.no_grad():
        save_last_hidden(chank_num, chank_size)
        

251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286


KeyboardInterrupt: 

In [12]:
%%time
with torch.no_grad():
    save_last_hidden(251, 10)

Wall time: 12.1 s
